In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores.faiss import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore(".cache/")

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 1. 문서 로드하기
loader = TextLoader("./files/document.txt")

# 2. 문서 쪼개기
docs = loader.load_and_split(text_splitter=splitter)

# 3. 임베딩 생성 및 캐시
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 4. 벡터 스토어 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

# 5. 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n
            Context: {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 6. 체인에 메모리 연결
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


# 체인 작동
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


# 메모리 확인
def get_history():
    return memory.load_memory_variables({})

In [3]:
get_history()

{'history': []}

In [4]:
invoke_chain("Is Aaronson guilty?")

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [5]:
invoke_chain("What message did he write in the table?")

content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.'


In [6]:
invoke_chain("Who is Julia?")

content='Julia is a character in the document who was involved with Winston in a forbidden relationship.'


In [7]:
invoke_chain("Do you remember what question did I ask?")

content='Yes, you asked about Julia, a character in the document.'


In [8]:
get_history()

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character in the document who was involved with Winston in a forbidden relationship.'),
  HumanMessage(content='Do you remember what question did I ask?'),
  AIMessage(content='Yes, you asked about Julia, a character in the document.')]}

In [9]:
load_memory({})

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character in the document who was involved with Winston in a forbidden relationship.'),
 HumanMessage(content='Do you remember what question did I ask?'),
 AIMessage(content='Yes, you asked about Julia, a character in the document.')]